<a href="https://colab.research.google.com/github/caltdreamer/GNN/blob/main/Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
!pip install -q torch-geometric
!pip install ogb
!pip install umap-learn
!pip install conditionalconformal

In [ ]:
import torch
import math
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GraphSAGE
from torch_geometric.data import NeighborSampler
from torch_geometric.nn import SAGEConv
from torch_geometric.datasets import Planetoid
from torch_geometric.datasets import NELL
from torch_geometric.datasets import CitationFull
from torch_geometric.datasets import CoraFull
from torch_geometric.datasets import Coauthor
from torch_geometric.data import DataLoader
from torch_geometric.nn import BatchNorm
from conditionalconformal.condconf import setup_cvx_problem_calib
import matplotlib.pyplot as plt
from tqdm import tqdm
import cvxpy as cp
import torch.optim as optim
import numpy as np
import os.path as osp
import pandas as pd
import numpy as np
import collections
import random
import networkx as nx
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neighbors import NearestNeighbors
from conditionalconformal.synthetic_data import generate_cqr_data, indicator_matrix
from conditionalconformal import CondConf

from scipy.stats import norm
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from tqdm import tqdm

In [ ]:
dataset = CoraFull(root='./data')
#dataset = Planetoid(root='./data', name='CiteSeer',split = 'public')
#dataset = Coauthor(root = './data',name ='CS' )
data = dataset[0]
print(data)
print(data.x)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
x = data.x.to(device)
y = data.y.squeeze().to(device)


In [ ]:
#Randomly choose train and calitest set
marginal_condi = []
marginal_split = []
prediction_split = []
prediction_condi=[]
test_index=[]
train_index=[]
calib_index=[]
for c in range(10):
      train_percentage = 0.6
      test_percentage = 0.4



      num_nodes = data.num_nodes
      all_node_indices = list(range(num_nodes))
      random.shuffle(all_node_indices)
      num_train_nodes = int(train_percentage * num_nodes)
      num_testandcali_nodes = num_nodes - num_train_nodes
      train_node_indices = all_node_indices[:num_train_nodes]
      testandcali_node_indices = all_node_indices[num_train_nodes:]

      #print(testandcali_node_indices)
      test_node_indices = testandcali_node_indices[:int(test_percentage * num_testandcali_nodes)]
      cali_node_indices = testandcali_node_indices[int(test_percentage * num_testandcali_nodes):]

      #print(test_node_indices)

      train_mask = torch.zeros(num_nodes, dtype=torch.bool)
      train_mask[train_node_indices] = True
      train_mask = torch.zeros(num_nodes, dtype=torch.bool)
      train_mask[train_node_indices] = True

      testandcali_mask = torch.zeros(num_nodes,dtype=torch.bool)
      testandcali_mask[testandcali_node_indices] = True

      test_mask = torch.zeros(num_nodes,dtype=torch.bool)
      test_mask[test_node_indices] = True


      cali_mask = torch.zeros(num_nodes,dtype=torch.bool)
      cali_mask[cali_node_indices] = True
      #***************************************************
      train_index.append(train_mask)
      calib_index.append(cali_mask)
      test_index.append(test_mask)
      #***************************************************
      train_loader = NeighborSampler(
        data.edge_index, node_idx=train_mask,
        sizes=[5,5,5], batch_size=32, shuffle=False,
      )
      #***************************************************
      class SAGE(torch.nn.Module):
        def __init__(self,in_channels,hidden_channels,out_channels,num_layers = 3):
            super(SAGE,self).__init__()

            self.numlayers = num_layers
            self.convs = torch.nn.ModuleList()
            self.convs.append(SAGEConv(in_channels,hidden_channels))
            for i in range(num_layers - 2):
                self.convs.append(SAGEConv(hidden_channels,hidden_channels))
            self.convs.append(SAGEConv(hidden_channels,out_channels))
        def reset_parameters(self):
            for conv in self.convs:
                conv.reset_parameters()
        def forward(self,x,adjs):
            for i ,(edge_index,_,size) in enumerate(adjs):
                xs = []
                x_target = x[:size[1]]#x should be a list prioritizing target nodes
                x = self.convs[i]((x,x_target),edge_index)
                if i != self.numlayers -1:
                    x = F.relu(x)
                    x = F.dropout(x,p=0.5,training=self.training)
                xs.append(x)
                if i == 0:
                    x_all = torch.cat(xs, dim=0)
                    layer_1_embeddings = x_all
                elif i == 1:
                    x_all = torch.cat(xs, dim=0)
                    layer_2_embeddings = x_all
                elif i == 2:
                    x_all = torch.cat(xs, dim=0)
                    layer_3_embeddings = x_all
            return layer_1_embeddings, layer_2_embeddings, layer_3_embeddings

        def inference(self, x,adjs):
            for i ,(edge_index,_,size) in enumerate(adjs):
                xs = []
                x_target = x[:size[1]]#x should be a list prioritizing target nodes
                x = self.convs[i]((x,x_target),edge_index)
                if i != self.numlayers -1:
                    x = F.relu(x)
                    x = F.dropout(x,p=0.5,training=self.training)
                xs.append(x)
                if i == 0:
                    x_all = torch.cat(xs, dim=0)
                    layer_1_embeddings = x_all
                elif i == 1:
                    x_all = torch.cat(xs, dim=0)
                    layer_2_embeddings = x_all
                elif i == 2:
                    x_all = torch.cat(xs, dim=0)
                    layer_3_embeddings = x_all
            return layer_1_embeddings, layer_2_embeddings, layer_3_embeddings
      #***************************************************
      model = SAGE(dataset.num_features, 128, dataset.num_classes, num_layers=3)

      model.reset_parameters()
      #We only input the initializing variables
      model = model.to(device)
      #***************************************************
      def train(epoch):
        model.train()
        total_loss = total_correct =0

        for batch_size,n_id,adjs in train_loader:
            adjs = [adj.to(device) for adj in adjs]
            optimizer.zero_grad()
            l1_emb, l2_emb, l3_emb = model(x[n_id], adjs)
            out = l3_emb.log_softmax(dim=-1)
            loss = F.nll_loss(out,y[n_id[:batch_size]])
            loss.backward()
            optimizer.step()

            total_loss += float(loss)
            total_correct += int(out.argmax(dim=-1).eq(y[n_id[:batch_size]]).sum())


        loss = total_loss / len(train_loader)


        approx_acc = total_correct / int(train_mask.sum())

        return loss, approx_acc
      #***************************************************
      optimizer = torch.optim.Adam(model.parameters(),lr = 0.001)

      for epoch in range(1,10):
        loss,acc = train(epoch)
        print(f'Epoch {epoch:02d}, Loss: {loss:.4f}, Approx. Train: {acc:.4f}')
      #***************************************************

      CaliGraph_loader = NeighborSampler(data.edge_index,node_idx = cali_mask
      , sizes = [5,5,5],batch_size = 10000,shuffle=False)

      TestGraph_loader  = NeighborSampler(data.edge_index,node_idx = test_mask
      , sizes = [5,5,5],batch_size = 10000,shuffle=False)
      #***************************************************
      #function used to output final layer embedding and predicted labels
      def inference_1(subgraph_loader):
        model.eval()
        total_loss = total_correct = 0
        for batch_size,n_id,adjs in subgraph_loader:

            emb1,emb2,emb3 = model(x[n_id],adjs)
            out = emb3.log_softmax(dim=-1)
            y_pred = out.argmax(dim=-1,keepdim =True)
            c = x[n_id][:batch_size]
            fn = torch.cat((c, emb3.softmax(dim=-1)), dim=1)
            return fn,emb3, y_pred
      #***************************************************
      #Use this one

      fn_cali,emb3_Cali,y_pred_Cali = inference_1(CaliGraph_loader)
      emb3_Cali = emb3_Cali.softmax(dim=-1)
      numpy_emb3_Cali = emb3_Cali.detach().cpu().numpy()
      print(numpy_emb3_Cali[0:10])
      #row_sums = np.sum(numpy_emb3_Cali, axis=1).reshape(-1, 1)
      #emb3_Cali = numpy_emb3_Cali/row_sums


      numpy_y_pred_Cali = y_pred_Cali.detach().cpu().numpy()
      numpy_y_labels_Cali = y[cali_mask].detach().cpu().numpy()
      numpy_y_labels_Test = y[test_mask].detach().cpu().numpy()
      numpy_y_labels_column = np.array(numpy_y_labels_Cali).reshape(-1, 1)
      #accuracy of our model for calibration nodes
      acc = (numpy_y_pred_Cali == numpy_y_labels_column).sum()/cali_mask.sum()

      alpha = 0.05
      n=cali_mask.sum()
      m=test_mask.sum()

      #Using APS score function
      cal_smx_tensor = torch.FloatTensor(emb3_Cali)
      cal_smx = cal_smx_tensor.detach().cpu().numpy()
      cal_pi = cal_smx.argsort(1)[:, ::-1]
      cal_srt = np.take_along_axis(cal_smx,cal_pi,axis=1).cumsum(axis=1)
      cal_scores = np.take_along_axis(cal_srt, cal_pi.argsort(axis=1), axis=1)[
        range(n), numpy_y_labels_Cali
      ]

      qhat = np.quantile(
        cal_scores, np.ceil((n + 1) * (1 - alpha)) / n, interpolation="higher"
      )

      #Use this one


      fn_test,emb3_Test,y_pred_Test = inference_1(TestGraph_loader)
      emb3_Test = emb3_Test.softmax(dim=-1)
      numpy_emb3_Test = emb3_Test.detach().cpu().numpy()
      test_smx = numpy_emb3_Test
      #row_sums = np.sum(numpy_emb3_Test, axis=1).reshape(-1, 1)
      #test_smx = numpy_emb3_Test/row_sums
      test_pi = test_smx.argsort(1)[:, ::-1]
      test_srt = np.take_along_axis(test_smx,test_pi,axis=1).cumsum(axis=1)

      test_scores = np.take_along_axis(test_srt, test_pi.argsort(axis=1), axis=1)[
        range(m), numpy_y_labels_Test
      ]


      prediction_sets = np.take_along_axis(test_srt <= qhat, test_pi.argsort(axis=1), axis=1)

      #***************************************************
      print(prediction_sets)
      empirical = prediction_sets[
        np.arange(prediction_sets.shape[0]), numpy_y_labels_Test
      ].mean()

      print(f"The empirical marginal coverage is: {empirical}")
      #***************************************************
      #Now use CondConf to predict the interval
      ### our conditional calibration method cover the test points
      def computeCoverages(XCal, scoresCal, XTest, scoresTest, alpha):

        coveragesCond = np.zeros(len(XTest))
        for i in tqdm(range(len(XTest))):
            prob = setup_cvx_problem_calib(1-alpha,None,
                                                  np.concatenate((scoresCal,np.array([scoresTest[i]]))), np.vstack((XCal,XTest[i,:])),{})
            prob.solve()
            #print(prob.constraints[2].shape)
            coveragesCond[i] = scoresTest[i] <= XTest[i,:]@prob.constraints[2].dual_value

        return coveragesCond

      coveragesCond = computeCoverages(numpy_emb3_Cali, cal_scores,numpy_emb3_Test, test_scores, alpha = 0.05)
      print(f"The conditional coverage is: {coveragesCond.mean()}")

      #print(f"The conditional coverage is: {coveragesCond.mean()}")
      marginal_condi.append(coveragesCond.mean())
      marginal_split.append(empirical)
      prediction_split.append(prediction_sets)
      prediction_condi.append(coveragesCond)
      #***************************************************




In [ ]:

def serialize_ndarrays(list_of_ndarrays):
    return '\n\n'.join('\n'.join(' '.join(str(cell) for cell in row) for row in arr) if arr.ndim > 1 else ' '.join(str(cell) for cell in arr) for arr in list_of_ndarrays)

# Assuming your list of tensors is named list_of_tensors

list_of_numpy_arrays_train = [tensor.numpy() for tensor in train_index if tensor.requires_grad == False]
list_of_numpy_arrays_calib = [tensor.numpy() for tensor in calib_index if tensor.requires_grad == False]
list_of_numpy_arrays_test = [tensor.numpy() for tensor in test_index if tensor.requires_grad == False]

# Assuming your list of ndarray of booleans is named list_of_boolean_arrays
train_index_int = [arr.astype(int) for arr in list_of_numpy_arrays_train]
calib_index_int = [arr.astype(int) for arr in list_of_numpy_arrays_calib]
test_index_int = [arr.astype(int) for arr in list_of_numpy_arrays_test]

train_index_string = serialize_ndarrays(train_index)
calib_index_string = serialize_ndarrays(calib_index)
test_index_string = serialize_ndarrays(test_index)
prediction_set_string =serialize_ndarrays(prediction_sets)
prediction_condi_string = serialize_ndarrays(prediction_condi)
#marginal_condi_string = serialize_ndarrays(marginal_condi)
#marginal_split_string = serialize_ndarrays(marginal_split)

file_path = '/content/drive/My Drive/Classification_Chicago_10runs.txt'
separator = "\n" + "-" * 50 + "\n"  # Creates a line of dashes as a separator.

with open(file_path, 'w') as f:
     f.write(train_index_string + separator)
     #f.write(calib_index_string + separator)
    # f.write(test_index_string + separator)
   # f.write(marginal_condi_string + separator)
   # f.write(marginal_split_string + separator)
     f.write(prediction_set_string + separator)
     f.write(prediction_condi_string)


In [48]:


numbers_string_condi = ' '.join(str(number) for number in marginal_condi)
numbers_string_split = ' '.join(str(number) for number in marginal_split)
with open(file_path, 'w') as f:
     f.write(numbers_string_condi + separator)
     #f.write(calib_index_string + separator)
    # f.write(test_index_string + separator)
   # f.write(marginal_condi_string + separator)
   # f.write(marginal_split_string + separator)
     f.write(numbers_string_split + separator)

